In [1]:
import Titanic
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline

### combining the readied file with no null ages, adding the survived column with null ages for the data that was from the test set

In [2]:

df = pd.read_csv('ready1.csv')
df2 = pd.read_csv('train.csv')
column = df2[['Survived']]
df['Survived'] = pd.Series(np.round(df2['Survived'].values, 0))

train = df.loc[df['Survived'].notnull()]
test = df.loc[df['Survived'].isnull()]
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,TicketFreq,Title,Deck,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1,Mr,U,0.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,Mrs,C,1.0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,Miss,U,1.0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,Mrs,C,1.0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,Mr,U,0.0


In [3]:

features = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'TicketFreq', 'Title', 'Deck']
category  = ['Pclass', 'Sex', 'Embarked','Title','Deck']
numerical = ['PassengerId','Age','SibSp','Parch','Fare', 'TicketFreq']
clt = make_column_transformer(
    # (StandardScaler(), numerical),
    (OneHotEncoder(), category),
                             remainder = 'passthrough')


## Using LogisticRegression

In [4]:
from sklearn.linear_model import LogisticRegression

X = train.drop(columns=['Name','Survived'])
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
# cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()
log = LogisticRegression(random_state=42, max_iter = 10000)
pipe = make_pipeline(clt, log)
param_grid = dict(
    logisticregression__class_weight =  [None, 'balanced', {0: 0.3, 1: 0.7}, {0: 0.2, 1: 0.8}],
    logisticregression__C = [0.01, 0.1, 1, 10, 100],
    logisticregression__penalty = ['l1', 'l2'],
    logisticregression__solver = ['liblinear'],  # supports both L1 and L2
)
grid = GridSearchCV(pipe,
                    param_grid = param_grid,
                   cv = 10, 
                   scoring = 'accuracy'
                   )
grid.fit(X_train,y_train)


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'logisticregression__C': [0.01, 0.1, ...], 'logisticregression__class_weight': [None, 'balanced', ...], 'logisticregression__penalty': ['l1', 'l2'], 'logisticregression__solver': ['liblinear']}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,10
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('onehotencoder', ...)]"


In [5]:
probs = grid.predict_proba(X_test)[:, 1]
y_pred = (probs >= 0.6).astype(int)
print(y_pred)
metrics.accuracy_score(y_test, y_pred)

[1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1
 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0]


0.8212290502793296

In [6]:
y_pred = grid.predict(X_test)
# probs = grid.predict_proba(X_test)[:, 1]
# y_pred = (probs >= 0.5).astype(int)
print(y_pred)
metrics.accuracy_score(y_test, y_pred)

[1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0.]


0.8156424581005587

### Predicting the survived using logisticregression

In [18]:
X = test[features]
probs = grid.predict_proba(X)[:, 1]
survived = (probs >= 0.6).astype(int)
test.loc[test['Survived'].isnull(), 'Survived'] = survived
test['Survived'] = test['Survived'].astype('Int64')
submit = test[['PassengerId', 'Survived']]
submit.to_csv('ready3.csv', index=False)

/var/folders/s6/bg6qmvhn05n0dz26ypf8py8r0000gp/T/ipykernel_4213/2727248670.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Survived'] = test['Survived'].astype('Int64')


## Using KneighborsClassifier 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
pipe = make_pipeline(clt, knn)
# cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

grid = GridSearchCV(pipe,
                    param_grid = {'kneighborsclassifier__n_neighbors':range(1,10)},
                   cv = 10, 
                   scoring = 'accuracy'
                   )
grid.fit(X,y)
apd.DataFrame(grid.cv_results_)